# Project Group 2

## Import libraries

In [101]:
import pandas as pd
import numpy as np
from pathlib import Path

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

import matplotlib.pyplot as plt
import seaborn as sns

In [133]:
driver = webdriver.Chrome()


driver.get("https://airbnb.com")

sleep(5)

try:
    driver.find_element(
        By.XPATH,
        value="/html/body/div[5]/div/div/div[1]/div/div[6]/section/div[2]/div[2]/button",
    ).click()
except:
    pass

In [134]:
city = "Valencia"
# arrival = "4 oct."
# leaving = "6 oct."

In [135]:
driver.find_element(
    By.ID,
    value="bigsearch-query-location-input",
).send_keys(city)

In [136]:
driver.find_element(
    By.CLASS_NAME,
    value="bhtghtc",
).click()

In [137]:
# Store the ID of the original window
original_window = driver.current_window_handle

In [138]:
driver.find_element(
    By.XPATH,
    value="/html/body/div[5]/div/div/div[1]/div/div[3]/div[2]/main/div[2]/div/div[2]/div/div/div/div/div[1]/div[1]",
).click()

In [139]:
driver.switch_to.window(driver.window_handles[1])

In [140]:
try:
    driver.find_element(
        By.XPATH,
        value="/html/body/div[9]/div/div/section/div/div/div[2]/div/div[1]/button",
    ).click()
except:
    pass

In [141]:
# Close the tab or window
driver.close()